# Homework 3: Due 11/20
In this assignment, we will be building a Naïve Bayes classifier and a SVM model for the productivity satisfaction of [the given dataset](https://archive.ics.uci.edu/ml/datasets/Productivity+Prediction+of+Garment+Employees), the productivity of garment employees.

## Background
The Garment Industry is one of the key examples of the industrial globalization of this modern era. It is a highly labour-intensive industry with lots of manual processes. Satisfying the huge global demand for garment products is mostly dependent on the production and delivery performance of the employees in the garment manufacturing companies. So, it is highly desirable among the decision makers in the garments industry to track, analyse and predict the productivity performance of the working teams in their factories. 

## Dataset Attribute Information

1. **date**: Date in MM-DD-YYYY
2. **day**: Day of the Week
3. **quarter** : A portion of the month. A month was divided into four quarters
4. **department** : Associated department with the instance
5. **team_no** : Associated team number with the instance
6. **no_of_workers** : Number of workers in each team
7. **no_of_style_change** : Number of changes in the style of a particular product
8. **targeted_productivity** : Targeted productivity set by the Authority for each team for each day.
9. **smv** : Standard Minute Value, it is the allocated time for a task
10. **wip** : Work in progress. Includes the number of unfinished items for products
11. **over_time** : Represents the amount of overtime by each team in minutes
12. **incentive** : Represents the amount of financial incentive (in BDT) that enables or motivates a particular course of action.
13. **idle_time** : The amount of time when the production was interrupted due to several reasons
14. **idle_men** : The number of workers who were idle due to production interruption
15. **actual_productivity** : The actual % of productivity that was delivered by the workers. It ranges from 0-1.

### Libraries that can be used: numpy, scipy, pandas, scikit-learn, cvxpy, imbalanced-learn
Any libraries used in the discussion materials are also allowed.

#### Other Notes

 - Don't worry about not being able to achieve high accuracy, it is neither the goal nor the grading standard of this assignment. <br >
 - If not specified, you are not required to do hyperparameter tuning, but feel free to do so if you'd like.

#### Trouble Shooting
In case you have trouble installing and using imbalanced-learn(imblearn) <br >
Run the below code cell, then go to the selection bar at top: Kernel > Restart. <br >
Then try `import imblearn` to see if things work. 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install imbalanced-learn delayed

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Exercises

## Exercise 1 - General Data Preprocessing (20 points)

Our dataset needs cleaning before building any models. Some of the cleaning tasks are common in general, but depends on what kind of models we are building, sometimes we have to do additional processing. These additional tasks will be mentioned in each of the remaining two exercises later.

Note that **we will be using this processed data from exercise 1 in each of the remaining two exercises**.

For convenience, here are the attributes that we would treat them as **categorical attributes**: `day`, `quarter`, `department`, and `team`. 

 - Drop the column `date`.
 - For each of the categorical attributes, **print out** all the unique elements.
 - For each of the categorical attributes, remap the duplicated items, if you find there are typos or spaces among the duplicated items.
     - For example, "a" and "a " should be the same, so we need to update "a " to be "a".
     - Another example, "apple" and "appel" should be the same, so you should update "appel" to be "apple".
     

 - Create another column named `satisfied` that records the productivity performance. The behavior defined as follows. **This is the dependent variable we'd like to classify in this assignment.**
     - Return True or 1 if `actual_productivity` is equal to or greater than `targeted_productivity`. Otherwise, return False or 0, which means the team fails to meet the expected performance.
 - Drop the columns `actual_productivity` and `targeted_productivity`.


 - Find and **print out** which columns/attributes that have empty vaules, e.g., NA, NaN, null, None.
 - Fill the empty values with 0.


In [3]:
# If put the data(.csv) under the same folder, you could use
import pandas as pd
df = pd.read_csv('./garments_worker_productivity.csv')

In [4]:
df = df.drop(columns=['date'])
df.head()

,quarter,department,day,team,targeted_productivity,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,actual_productivity
0,Quarter1,sweing,Thursday,8,0.80,26.16,1108.0,7080,98,0.0,0,0,59.0,0.940725
1,Quarter1,finishing,Thursday,1,0.75,3.94,NaN,960,0,0.0,0,0,8.0,0.886500
2,Quarter1,sweing,Thursday,11,0.80,11.41,968.0,3660,50,0.0,0,0,30.5,0.800570
3,Quarter1,sweing,Thursday,12,0.80,11.41,968.0,3660,50,0.0,0,0,30.5,0.800570
4,Quarter1,sweing,Thursday,6,0.80,25.90,1170.0,1920,50,0.0,0,0,56.0,0.800382


In [5]:
cats = ['day', 'quarter', 'department', 'team']
for var in cats:
  print(df[var].unique())

['Thursday' 'Saturday' 'Sunday' 'Monday' 'Tuesday' 'Wednesday']
['Quarter1' 'Quarter2' 'Quarter3' 'Quarter4' 'Quarter5']
['sweing' 'finishing ' 'finishing']
[ 8  1 11 12  6  7  2  3  9 10  5  4]


There are duplicates in the department category, we will clean these up.

In [6]:
df['department'] = df['department'].map({'sweing': 'sewing', 'finishing ': 'finishing', 'finishing':'finishing'})

In [7]:
df.department.unique()

array(['sewing', 'finishing'], dtype=object)

There is now only 'sewing' and 'finishing', so it looks like our processing worked.

In [8]:
# checking that team is int rather than string, they are so we will move on
[type(x) for x in df.team.unique()]

[numpy.int64,
 numpy.int64,
 numpy.int64,
 numpy.int64,
 numpy.int64,
 numpy.int64,
 numpy.int64,
 numpy.int64,
 numpy.int64,
 numpy.int64,
 numpy.int64,
 numpy.int64]

Now we will create the 'satisfied' column.

In [9]:
satisfied = lambda x: int( x['actual_productivity'] >= x['targeted_productivity'] )
df['satisfied'] = df.apply(satisfied, axis=1)

In [10]:
df = df.drop(columns=['actual_productivity', 'targeted_productivity'])

Now, we will replace all null or NaN values with 0.

In [11]:
df.isna().any()

quarter               False
department            False
day                   False
team                  False
smv                   False
wip                    True
over_time             False
incentive             False
idle_time             False
idle_men              False
no_of_style_change    False
no_of_workers         False
satisfied             False
dtype: bool

Only the 'wip' column has missing values, so we only need to target that column.

In [12]:
import numpy as np # could also use the smaller math library, but we will be using np anyways
df['wip'] = df['wip'].map(lambda x: 0 if np.isnan(x) else x)

In [13]:
df.isna().any()

quarter               False
department            False
day                   False
team                  False
smv                   False
wip                   False
over_time             False
incentive             False
idle_time             False
idle_men              False
no_of_style_change    False
no_of_workers         False
satisfied             False
dtype: bool

Now that we have resolved all null values, we have completed our general data preprocessing.

## Exercise 2 - Naïve Bayes Classifier (40 points in total)

### Exercise 2.1 - Additional Data Preprocessing (10 points)

To build a Naïve Bayes Classifier, we need to further encode our categorical variables.

 - For each of the **categorical attribtues**, encode the set of categories to be **0 ~ (n_classes - 1)**.
     - For example, \["paris", "paris", "tokyo", "amsterdam"\] should be encoded as \[1, 1, 2, 0\].
     - Note that the order does not really matter, i.e., \[0, 0, 1, 2\] also works. But you have to start with 0 in your encodings.
     - You can find information about this encoding in the discussion materials.


 - Split the data into training and testing set with the ratio of 80:20.

In [14]:
# Remember to continue the task with your processed data from Exercise 1
# print(f'categorical variables: {cats}\n')

# define encoding
gen_encode = lambda unique: dict( zip( sorted(unique), range(len(unique)) ) )

encoding = {} # save encoding for later reference
# encoding = {'day': {'Sunday': 0, 'Monday': 1, 'Tuesday': 2, 'Wednesday': 3, 
                    # 'Thursday': 4, 'Friday': 5, 'Saturday': 6}} 

df_encode = df.copy()

def encode(category, value):
  return encoding[category][value]

def decode(category, code):
  return [key for key in encoding[category] if encoding[category][key] == code][0]

for c in cats:
  encoding[c] = gen_encode(df[c].unique()) # save to encoding
  df_encode[c] = df.apply( lambda row: encode(c, row[c]), axis=1) # perform encoding

In [15]:
from sklearn.model_selection import train_test_split
X = df_encode.drop(columns=['satisfied'])
y = df_encode['satisfied'].to_numpy().reshape(-1,1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.20, random_state=42)

### Exercise 2.2 - Naïve Bayes Classifier for Categorical Attributes (15 points)

Use the categorical attributes **only**, please build a Categorical Naïve Bayes classifier that predicts the column `satisfied`. <br >
Report the **testing result** using `classification_report`.

In [16]:
# Remember to do this task with your processed data from Exercise 2.1
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report

X_train_cat, X_test_cat = X_train[cats], X_test[cats]

gnb = GaussianNB()
y_pred = gnb.fit(X_train_cat, y_train.ravel()).predict(X_test_cat)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.17      0.02      0.03        63
           1       0.74      0.97      0.84       177

    accuracy                           0.72       240
   macro avg       0.45      0.49      0.43       240
weighted avg       0.59      0.72      0.62       240



### Exercise 2.3 - Naïve Bayes Classifier for Numerical Attributes (15 points)

Use the numerical attributes **only**, please build a Gaussian Naïve Bayes classifier that predicts the column `satisfied`. <br >
Report the **testing result** using `classification_report`.

**Remember to scale your data. The scaling method is up to you.**

In [17]:
# Remember to do this task with your processed data from Exercise 2.1
X_train_num, X_test_num = X_train.drop(columns=cats), X_test.drop(columns=cats)

from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler()
scaler.fit(X_train_num)


Z_train = pd.DataFrame(scaler.transform(X_train_num), columns=X_train_num.columns, index=X_train_num.index)
Z_test = pd.DataFrame(scaler.transform(X_test_num), columns=X_test_num.columns, index=X_test_num.index)

gnb = GaussianNB()
y_pred = gnb.fit(Z_train, y_train.ravel()).predict(Z_test)

print( classification_report(y_test, y_pred) )

              precision    recall  f1-score   support

           0       0.37      0.62      0.46        63
           1       0.82      0.63      0.71       177

    accuracy                           0.62       240
   macro avg       0.60      0.62      0.59       240
weighted avg       0.70      0.62      0.65       240



## Exercies 3 - SVM Classifier (40 points in total)

### Exercise 3.1 - Additional Data Preprocessing (10 points)

To build a SVM Classifier, we need a different encoding for our categorical variables.

 - For each of the **categorical attribtues**, encode them with **one-hot encoding**.
     - You can find information about this encoding in the discussion materials.


 - Split the data into training and testing set with the ratio of 80:20.


In [19]:
# Remember to continue the task with your processed data from Exercise 1

enc = preprocessing.OneHotEncoder()
enc.fit(X_train_cat)
X_train_cat_onehot = pd.DataFrame(enc.transform(X_train_cat).toarray(), index=X_train.index)
X_train_onehot = pd.concat([X_train_cat_onehot, Z_train], axis=1)

enc = preprocessing.OneHotEncoder()
enc.fit(X_test_cat)
X_test_cat_onehot = pd.DataFrame(enc.transform(X_test_cat).toarray(), index=X_test.index)
X_test_onehot = pd.concat([X_test_cat_onehot, Z_test], axis=1)

In [20]:
from sklearn.svm import SVC

svm = SVC(kernel='linear')
svm.fit(X_train_onehot, y_train.ravel())
y_pred = svm.predict(X_test_onehot)
print(classification_report(y_test.ravel(), y_pred.reshape(-1,1)))

              precision    recall  f1-score   support

           0       1.00      0.02      0.03        63
           1       0.74      1.00      0.85       177

    accuracy                           0.74       240
   macro avg       0.87      0.51      0.44       240
weighted avg       0.81      0.74      0.64       240



/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


### Exercise 3.2 - SVM with Different Kernels (20 points)

Using all the attributes we have, please build a SVM that predicts the column `satisfied`. <br >
Specifically, please 
 - Build one SVM with **linear kernel**.
 - Build another SVM but with **rbf kernel**.
 - Report the **testing results** of **both models** using `classification report`.

The kernel is the only setting requirement. <br >
Other hyperparameter tuning is not required. But make sure they are the same in these two SVMs if you'd like to tune the model. In other words, the only difference between the two SVMs should be the kernel setting.

**Remember to scale your data. The scaling method is up to you.**

In [21]:
# Remember to do this task with your processed data from Exercise 3.1
from sklearn.svm import SVC

svm = SVC(kernel='linear')
svm.fit(X_train_onehot, y_train.ravel())
y_pred = svm.predict(X_test_onehot)
print(classification_report(y_test.ravel(), y_pred.reshape(-1,1)))

              precision    recall  f1-score   support

           0       1.00      0.02      0.03        63
           1       0.74      1.00      0.85       177

    accuracy                           0.74       240
   macro avg       0.87      0.51      0.44       240
weighted avg       0.81      0.74      0.64       240



/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [22]:
from sklearn.svm import SVC

svm = SVC(kernel='rbf')
svm.fit(X_train_onehot, y_train.ravel())
y_pred = svm.predict(X_test_onehot)
print(classification_report(y_test.ravel(), y_pred.reshape(-1,1)))

              precision    recall  f1-score   support

           0       0.48      0.25      0.33        63
           1       0.77      0.90      0.83       177

    accuracy                           0.73       240
   macro avg       0.63      0.58      0.58       240
weighted avg       0.70      0.73      0.70       240



/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


### Exercise 3.3 - SVM with Over-sampling (10 points)
 - For the column `satisfied` in our **training set**, please **print out** the frequency of each class. 
 - Oversample the **training data**. 
 - For the column `satisfied` in the oversampled data, **print out** the frequency of each class  again.
 - Re-build the 2 SVMs with the same setting you have in Exercise 3.2, but **use oversampled training data** instead.
     - Do not forget to scale the data first. As always, the scaling method is up to you.
 - Report the **testing result** with `classification_report`.

You can use ANY methods listed on [here](https://imbalanced-learn.org/stable/references/over_sampling.html#) such as RandomOverSampler or SMOTE. <br > 
You are definitely welcomed to build your own oversampler. <br >

Note that you do not have to over-sample your testing data.

In [38]:
# Remember to do this task with your processed data from Exercise 3.1
print(f"Number of 1's: {y_train.sum()}")
print(f"Number of 0's: {y_train.size - y_train.sum()}")
print(f"Out of: {y_train.size}")


Number of 1's: 698
Number of 0's: 259
Out of: 957


In [39]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=21)
X_os, y_os = ros.fit_resample(X_train, y_train)

In [40]:
print(f"Number of 1's: {y_os.sum()}")
print(f"Number of 0's: {y_os.size - y_os.sum()}")
print(f"Out of: {y_os.size}")

Number of 1's: 698
Number of 0's: 698
Out of: 1396


In [52]:
# split data set
X_train, X_test, y_train, y_test = train_test_split(X_os, y_os, test_size=0.2, random_state=42)

# split categorical and numerical data
X_train_cat, X_test_cat = X_train[cats], X_test[cats]
X_train_num, X_test_num = X_train.drop(columns=cats), X_test.drop(columns=cats)

# scale numerical data
scaler = preprocessing.MinMaxScaler()
scaler.fit(X_train_num)

# scaled numerical data frame
Z_train = pd.DataFrame(scaler.transform(X_train_num), columns=X_train_num.columns, index=X_train_num.index)
Z_test = pd.DataFrame(scaler.transform(X_test_num), columns=X_test_num.columns, index=X_test_num.index)

# one hot encode cat vars
enc = preprocessing.OneHotEncoder()
enc.fit(X_train_cat)
X_train_cat_onehot = pd.DataFrame(enc.transform(X_train_cat).toarray(), index=X_train.index)
X_train_onehot = pd.concat([X_train_cat_onehot, Z_train], axis=1)

enc = preprocessing.OneHotEncoder()
enc.fit(X_test_cat)
X_test_cat_onehot = pd.DataFrame(enc.transform(X_test_cat).toarray(), index=X_test.index)
X_test_onehot = pd.concat([X_test_cat_onehot, Z_test], axis=1)

In [53]:
from sklearn.svm import SVC

svm = SVC(kernel='linear')
svm.fit(X_train_onehot, y_train.ravel())
y_pred = svm.predict(X_test_onehot)
print(classification_report(y_test.ravel(), y_pred.reshape(-1,1)))

              precision    recall  f1-score   support

           0       0.75      0.68      0.71       151
           1       0.66      0.73      0.69       129

    accuracy                           0.70       280
   macro avg       0.70      0.71      0.70       280
weighted avg       0.71      0.70      0.70       280



/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [54]:
from sklearn.svm import SVC

svm = SVC(kernel='rbf')
svm.fit(X_train_onehot, y_train.ravel())
y_pred = svm.predict(X_test_onehot)
print(classification_report(y_test.ravel(), y_pred.reshape(-1,1)))

              precision    recall  f1-score   support

           0       0.80      0.77      0.79       151
           1       0.74      0.77      0.76       129

    accuracy                           0.77       280
   macro avg       0.77      0.77      0.77       280
weighted avg       0.77      0.77      0.77       280



/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


## Exercise 4 - Collaborative Statement (5 points) 
### You must fill this out even if you worked alone to get credit.

It is mandatory to include a Statement of Collaboration in each submission, that follows the guidelines below.
Include the names of everyone involved in the discussions (especially in-person ones), and what was discussed.
All students are required to follow the academic honesty guidelines posted on the course website. For
programming assignments in particular, I encourage students to organize (perhaps using Piazza) to discuss the
task descriptions, requirements, possible bugs in the support code, and the relevant technical content before they
start working on it. However, you should not discuss the specific solutions, and as a guiding principle, you are
not allowed to take anything written or drawn away from these discussions (no photographs of the blackboard,
written notes, referring to Piazza, etc.). Especially after you have started working on the assignment, try to restrict
the discussion to Piazza as much as possible, so that there is no doubt as to the extent of your collaboration.

I read some posts on Piazza to get a better idea of how to go about joining the data frames after performing one hot encoding. I also read through the code provided in discussion jupyter notebooks to know which sklearn library functions to use.